In [ ]:
#6-1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [ ]:
class LineItem:
  """구매할 물품/갯수 생성해서 총 가격을 반환"""
  def __init__(self, product, quantity, price):
      self.product = product
      self.quantity = quantity
      self.price = price
  def total(self):
      return self.price * self.quantity

In [ ]:
class Order:
  def __init__(self, customer, cart, promotion=None):
      self.customer = customer
      self.cart = list(cart)
      self.promotion = promotion

  def total(self):
      if not hasattr(self, '__total'):
          self.__total = sum(item.total() for item in self.cart)
      return self.__total

  def due(self):
      if self.promotion is None:
          discount = 0
      else:
          discount = self.promotion.discount(self)
      return self.total() - discount

  def __repr__(self):
      fmt = '<Order total: {:.2f} due: {:.2f}>'
      return fmt.format(self.total(), self.due())

In [ ]:
class Promotion(ABC):
  @abstractmethod
  def discount(self, order):
    pass

In [ ]:
class FidelityPromo(Promotion):
  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  def discount(self, order):
     distinct_items = {item.product for item in order.cart}
     if len(distinct_items) >= 10:
       return order.total() * 0.07
     return 0

In [ ]:
#6-2
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
  LineItem('banana', 4, .5),
  LineItem('apple', 10, 1.5),
  LineItem('watermellon', 5, 5.0)
]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [ ]:
banana_cart = [
  LineItem('banana', 30, .5),
  LineItem('apple', 10, 1.5)
]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [ ]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [ ]:
#6-3
class Order:
  def __init__(self, customer, cart, promotion=None):
      self.customer = customer
      self.cart = list(cart)
      self.promotion = promotion
  def total(self):
      if not hasattr(self, '__total'):
          self.__total = sum(item.total() for item in self.cart)
      return self.__total

  def due(self):
      if self.promotion is None:
          discount = 0
      else:
          discount = self.promotion(self)
      return self.total() - discount

  def __repr__(self):
      fmt = '<Order total: {:.2f} due: {:.2f}>'
      return fmt.format(self.total(), self.due())

In [ ]:
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [ ]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [ ]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [ ]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [ ]:
#6-4
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)

In [ ]:
#6-5
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [ ]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [ ]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', '_i2', 'namedtuple', 'Customer', 'park', '_2', '_i3', 'LineItem', '_i4', '_i5', 'Order', '_i6', 'Promotion', '_i7', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i8', 'joe', 'ann', 'cart', '_8', '_i9', '_9', '_i10', '_i11', '_11', '_i12', '_i13', '_13', '_i14', '_14', '_i15', 'banana_cart', '_15', '_i16', 'long_order', '_16', '_i17', '_17', '_i18', '_i19', '_i20', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i21', '_21', '_i22', '_22', '_i23', '_23', '_i24', '_24', '_i25', 'promos', 'best_promo', '_i26', '_26', '_i27', '_27', '_i28', '_28', '_i29'])

In [ ]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [ ]:
#6-6
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [ ]:
#7-1
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [ ]:
@deco
def target():
  print('running target()')

target()

running inner()


In [ ]:
target

<function __main__.deco.<locals>.inner()>

In [ ]:
#7-2
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7e012b1641f0>)
running register(<function f2 at 0x7e012b167eb0>)
running main()
registry -> [<function f1 at 0x7e012b1641f0>, <function f2 at 0x7e012b167eb0>]
running f1()
running f2()
running f3()


In [ ]:
#7-3
promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  promos.append(fidelity_promo)
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return

@promotion
def best_promo(order):
  return max(promo(order) for promo in promos)

In [ ]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [ ]:
b = 6
f1(3)

3
6


In [ ]:
#7-5
b=6
def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)


3


UnboundLocalError: local variable 'b' referenced before assignment

In [ ]:
#7-6
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9


f3(3)

3
6


In [ ]:
b

9

In [ ]:
f3(3)

3
9


In [ ]:
b = 30
f3(3)

3
30


In [ ]:
#7-7
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [ ]:
avg = Averager()
avg(10)

10.0

In [ ]:
avg(11)

10.5

In [ ]:
avg(12)

11.0

In [ ]:
#7-8
def make_averager():
  series = []
  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)
  return averager

In [ ]:
avg = make_averager()
avg(10)

10.0

In [ ]:
avg(11)

10.5

In [ ]:
avg(12)

11.0

In [ ]:
#7-9
avg.__code__.co_varnames

('new_value', 'total')

In [ ]:
avg.__code__.co_freevars

('series',)

In [ ]:
avg.__closure__

(<cell at 0x7e012b5144f0: list object at 0x7e012b14b2c0>,)

In [ ]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [ ]:
# 7-10
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    count += 1
    total += new_value
    return total / count
  return averager

In [ ]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [ ]:
#7-11
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  return averager

In [ ]:
avg = make_averager()
avg(10)

10.0

In [ ]:
#7-12
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [ ]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12320828s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000088s] factorial(1) -> 1
[0.00004414s] factorial(2) -> 2
[0.00132687s] factorial(3) -> 6
[0.00138778s] factorial(4) -> 24
[0.00141281s] factorial(5) -> 120
[0.00144517s] factorial(6) -> 720
6! = 720


In [ ]:
#7-13
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12319022s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000068s] factorial(1) -> 1
[0.00002981s] factorial(2) -> 2
[0.00004957s] factorial(3) -> 6
[0.00006878s] factorial(4) -> 24
[0.00008868s] factorial(5) -> 120
[0.00011029s] factorial(6) -> 720
6! = 720


In [ ]:
factorial.__name__

'clocked'

In [ ]:
from os import name
#7-14
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [ ]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12323141s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000072s] factorial(1) -> 1
[0.00003839s] factorial(2) -> 2
[0.00190663s] factorial(3) -> 6
[0.00195479s] factorial(4) -> 24
[0.00198841s] factorial(5) -> 120
[0.00217676s] factorial(6) -> 720
6! = 720


In [ ]:
factorial.__name__

'factorial'